<a href="https://colab.research.google.com/github/cheyroseflammer/big-data-challenge/blob/main/auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-03 18:04:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2022-02-03 18:04:59 (1.31 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), inferSchema=True, sep=r'\t', dateFormat="yyyy-MM-dd HH:mm:ss")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
# Show and print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Count df record amount
df.count()

3514942

## Transform The Dataset 

In [ ]:
# Dropping null values from dataset
new_df=df.dropna(how="any")
new_df.show(truncate=False)
# Check new count of data set
new_df.count()

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

3514045

In [ ]:
# Get rid of any duplicate entries on product id
drop_df = new_df.dropDuplicates(["product_id"])
drop_df.show(truncate=False)
# Check new df count
drop_df.count()

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                |product_category|star_rating|helpful_votes|total_votes|vine|verified_purcha

762734

In [ ]:
# Creating review_id table
clean_df = drop_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
clean_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2IQFDM47H6T0Z|   46718580|9539751705|     621444381| 2014-11-06|
|R3UCYMK8M1TVXG|   23077055|B00006JPBB|     950430275| 2012-02-13|
|R35XBEALUENO50|   25167962|B00009WNYB|      78729146| 2005-06-01|
|R2YTAYV1I4BTT1|   49987447|B0000AQM5V|     369038089| 2003-12-29|
|R1QQF6VF4MOBUV|   34046056|B0000AX5XB|     118992240| 2013-06-13|
|R2FSBYF8IZCLZ0|   42837617|B0000AXQO9|     289972734| 2015-02-04|
|R156EVLS4YRVR7|   10170419|B0000AXVCB|     221562182| 2013-06-24|
|R32E56WU7MML09|   52010547|B0000AY9W6|     292698258| 2013-06-26|
|R1X1645I69BW1U|   22418020|B0000AZ8PG|      10757068| 2014-07-19|
|R38LQ7XUCXD0MC|   30440473|B0000UUU82|     124443238| 2014-10-13|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Check schema 
clean_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Convert date format
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col, unix_timestamp, to_date


review_df = clean_df.withColumn('review_date', to_date(unix_timestamp(col('review_date'), 'yyyy-MM-dd').cast("timestamp")))
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2IQFDM47H6T0Z|   46718580|9539751705|     621444381| 2014-11-06|
|R3UCYMK8M1TVXG|   23077055|B00006JPBB|     950430275| 2012-02-13|
|R35XBEALUENO50|   25167962|B00009WNYB|      78729146| 2005-06-01|
|R2YTAYV1I4BTT1|   49987447|B0000AQM5V|     369038089| 2003-12-29|
|R1QQF6VF4MOBUV|   34046056|B0000AX5XB|     118992240| 2013-06-13|
|R2FSBYF8IZCLZ0|   42837617|B0000AXQO9|     289972734| 2015-02-04|
|R156EVLS4YRVR7|   10170419|B0000AXVCB|     221562182| 2013-06-24|
|R32E56WU7MML09|   52010547|B0000AY9W6|     292698258| 2013-06-26|
|R1X1645I69BW1U|   22418020|B0000AZ8PG|      10757068| 2014-07-19|
|R38LQ7XUCXD0MC|   30440473|B0000UUU82|     124443238| 2014-10-13|
|R176IMHT0KKWQF|   26097374|B0000UUX9S|     959671175| 2013-05-13|
|R16XDOMI2BHI2L|   14648868|B000182DO2|     182521644| 2012-12

In [ ]:
# Re-check schema after date format change
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Creating products table
products_df = drop_df.select(["product_id", "product_title"])
# Check dataframe
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9539751705|Tennessee Heavy D...|
|B00006JPBB|Ai HWH806 Factory...|
|B00009WNYB|Megaman X Ultimat...|
|B0000AQM5V|  Coghlan's Rainsuit|
|B0000AX5XB|Raider BCS-8B Del...|
|B0000AXQO9|Sierra Internatio...|
|B0000AXVCB|Meguiars M4364 Bo...|
|B0000AY9W6|Camco 44412 Wheel...|
|B0000AZ8PG|NGK (1052) B6HS-1...|
|B0000UUU82|Lund 18063 Interc...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Creating customer_id table
customer_df = drop_df.select(["customer_id"])

In [ ]:
# Count the number of times each customer appeared in the dataset
import pyspark.sql.functions as func
customers = customer_df.groupBy("customer_id").count().select(func.col("customer_id").alias("customer_id"),func.col("count").alias("customer_count"))
customers.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11222629|             6|
|   13151091|             1|
|   10658892|             2|
|   15564120|             1|
|   10944478|             4|
|   25559985|            13|
|   10217380|            15|
|   17193739|             1|
|    3400412|             1|
|   10510214|             3|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Creating the vine_table
vine_table = drop_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2IQFDM47H6T0Z|          1|            0|          0|   N|
|R3UCYMK8M1TVXG|          5|            0|          0|   N|
|R35XBEALUENO50|          3|            0|          0|   N|
|R2YTAYV1I4BTT1|          4|            0|          0|   N|
|R1QQF6VF4MOBUV|          5|            0|          0|   N|
|R2FSBYF8IZCLZ0|          5|            0|          0|   N|
|R156EVLS4YRVR7|          5|            0|          0|   N|
|R32E56WU7MML09|          4|            0|          0|   N|
|R1X1645I69BW1U|          5|            0|          0|   N|
|R38LQ7XUCXD0MC|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Configure RDS
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/big_data_db"
config = {"user":"root", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Pushing tables to the database tables
review_df.write.jdbc(url=jdbc_url, table='review_df', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)